In [ ]:
!pip install openai chromadb sentence-transformers pdfplumber

In [ ]:
import openai
import chromadb
import pdfplumber
from sentence_transformers import SentenceTransformer
from google.colab import files  # For file uploads

# Initialize OpenAI API Key
openai.api_key = "YOUR_API_KEY"
# Load Embedding Model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB (Vector Database)
chroma_client = chromadb.PersistentClient(path="./resume_db")
collection = chroma_client.get_or_create_collection(name="resumes")

print("✅ Setup Complete")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Setup Complete


In [ ]:
# Upload multiple PDF resumes
uploaded_files = files.upload()

# Function to extract text from multiple PDFs
def extract_text_from_pdfs(uploaded_files):
    resume_texts = {}  # Dictionary to store extracted text
    for filename in uploaded_files.keys():
        with pdfplumber.open(filename) as pdf:
            text = " ".join([page.extract_text() for page in pdf.pages if page.extract_text()])
            resume_texts[filename] = text  # Store extracted text
            print(f"✅ Extracted text from {filename}")
    return resume_texts

# Extract text from uploaded resumes
resume_texts = extract_text_from_pdfs(uploaded_files)


Saving Shreya A -Resume.pdf to Shreya A -Resume (1).pdf
Saving Shreya Resume BA.pdf to Shreya Resume BA.pdf
Saving Monika clg cv.pdf to Monika clg cv.pdf
✅ Extracted text from Shreya A -Resume (1).pdf
✅ Extracted text from Shreya Resume BA.pdf
✅ Extracted text from Monika clg cv.pdf


In [ ]:
# Function to Add Resumes to Vector DB
def add_resumes_to_db(resume_texts):
    for i, (filename, text) in enumerate(resume_texts.items()): # Added enumerate to generate unique ids
        candidate_name = filename.replace(".pdf", "")  # Use filename as candidate name
        embedding = embedding_model.encode(text).tolist()
        collection.add(
            ids=[str(i)],  # Provide unique IDs for each document
            embeddings=[embedding],
            documents=[text],
            metadatas=[{"name": candidate_name}],
        )
        print(f"📂 Stored {candidate_name} in Vector DB ✅")

# Add all extracted resumes to the database
add_resumes_to_db(resume_texts)


📂 Stored Shreya A -Resume (1) in Vector DB ✅
📂 Stored Shreya Resume BA in Vector DB ✅
📂 Stored Monika clg cv in Vector DB ✅


In [ ]:
# Function to Retrieve Matching Resumes
def search_resumes(job_description):
    job_embedding = embedding_model.encode(job_description).tolist()
    results = collection.query(query_embeddings=[job_embedding], n_results=3)  # Top 3 matches
    return results["documents"][0] if results["documents"] else []

# Example: Find candidates for a sample job
job_desc = "Looking for a Python developer with experience in machine learning."
matching_resumes = search_resumes(job_desc)

print("\n🔍 Top Matching Resumes Found:")
for i, resume in enumerate(matching_resumes):
    print(f"{i+1}. {resume[:200]}...")  # Show preview of resume text



🔍 Top Matching Resumes Found:
1. Shreya A
Bachelor of Technology - Computer Science and Engineering
Mail ID: shreyaanand555@gmail.com
Phone number: 7829956039
EDUCATION SKILLS
Alliance College of Engineering, Bengaluru Programming La...
2. MONIKA P
Office of Career Advancement and Networking, Alliance University,
Chandapura-Anekal Main Road, Bengaluru, 562106
Mob: +91-6362722581
Linkedin ID: Monika P | Linkedin
Email ID: pmonikabtech21@...
3. Shreya A
Police Qut, Kudlu, Bengaluru - 560100
Mob: +91-7829956039
Linkedin ID: Shreya A | LinkedIn
Email ID: ashreyabtech21@ced.alliance.edu.in
P
CAREER OBJECTIVE
To learn and gain new experience whi...


In [ ]:
import time

# Function to Generate AI Match Summary
def match_candidates(job_description):
    relevant_resumes = search_resumes(job_description)
    context = "\n".join(relevant_resumes)

    # Changed the model to 'gpt-3.5-turbo'
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo", # Switched to a more widely available model
            messages=[
                {"role": "system", "content": "You are an AI recruiter assistant. Match the job description with best candidates."},
                {"role": "user", "content": f"Job Description: {job_description}\nRelevant Resumes: {context}"}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except openai.error.RateLimitError:
        print("Rate limit exceeded. Waiting for 60 seconds...")
        time.sleep(60)  # Wait for 60 seconds before retrying
        return match_candidates(job_description) # Retry the request
